In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
crop_yield = pd.read_csv("./raw_data/fao_data/FAOSTAT_data_en_2-9-2023.csv")
change_temp = pd.read_csv("./raw_data/fao_data/FAOSTAT_data_en_2-9-2023 (1).csv")
precip = pd.read_csv("./raw_data/precipitation/observed_average_annual_precipitation.csv")
air_temp_mean = pd.read_csv("./raw_data/air_temperature/average_air_temperature.txt", sep=";", index_col=False)
frost_days = pd.read_csv("./raw_data/frost_days/regional_averages_tnas_year.txt", sep=";", skiprows=1)
hot_days = pd.read_csv("./raw_data/hot_days/regional_averages_txbs_year.txt", sep=";", skiprows=1)
ice_days = pd.read_csv("./raw_data/ice_days/regional_averages_txcs_year.txt", sep=";", skiprows=1)
summer_days = pd.read_csv("./raw_data/summer_days/regional_averages_txas_year.txt", sep=";", skiprows=1)#, index_col=False) 
sunshine_duration = pd.read_csv("./raw_data/sunshine_duration/regional_averages_sd_year.txt", sep=";", skiprows=1) # hours
tropical_nights = pd.read_csv("./raw_data/tropical_nights/regional_averages_tnes_year.txt", sep=";",skiprows=1)

### Creating the working set (FAO Data)

In [ ]:
crop_data_official = crop_yield.loc[(crop_yield["Flag"] == "A") | (crop_yield["Flag"] == "E")]
crop_data_official = crop_yield.loc[(crop_yield["Element"] == "Yield")]
crop_data_official = crop_data_official.reset_index(drop=True)
crop_data_official = crop_data_official.drop_duplicates()
crop_data_official.shape

In [ ]:
temp_data = change_temp
temp_change_data = temp_data[temp_data["Element"] == "Temperature change"]
temp_change_data.shape

In [ ]:
# Combines both dataframes and creates the final FAO dataframe
final_fao = crop_data_official[["Year", "Flag Description", "Element", "Item", "Value", "Unit"]] \
.merge(temp_change_data[["Year", "Flag Description", "Element", "Value", "Unit"]], left_on="Year", right_on="Year", how="inner")
final_fao

In [ ]:
final_fao[(final_fao["Year"] == 2010)].sample(10)# & final_data["Item"] == "Wheat"]#.iloc[0]

In [ ]:
final_fao = final_fao.drop("Flag Description_y", axis=1)
final_fao = final_fao.rename(columns={"Flag Description_x":"Flag Description", "Value_x":"Value", 
                                      "Unit_x":"Unit", "Element_x": "Food Element", "Element_y":"Temperature Element", 
                                      "Value_y":"Degrees", "Unit_y":"Temp Unit"})

In [ ]:
final_fao

### Creating the working Subset (DWD Data)

In [ ]:
de_pre = precip[["Category", "Annual Mean"]]
de_temp = air_temp_mean[["Jahr", "Deutschland"]]
de_frost = frost_days[["Jahr", "Deutschland"]]
de_hot = hot_days[["Jahr", "Deutschland"]]
de_ice = ice_days[["Jahr", "Deutschland"]]
de_sum = summer_days[["Jahr", "Deutschland"]]
de_sun = sunshine_duration[["Jahr", "Deutschland"]]
de_trop = tropical_nights[["Jahr", "Deutschland"]]

master_data = [de_pre, de_temp, de_frost, de_hot, de_ice, de_sum, de_sum, de_trop]

In [ ]:
sunshine_duration["modified"] = sunshine_duration["Deutschland"].apply(lambda x: (x / 24) / 365)
sunshine_duration = sunshine_duration["modified"]

In [ ]:
sunshine_duration

In [ ]:
plt.figure(figsize=(7, 7))
heatmap = sns.heatmap(final_data.corr(method='kendall'), annot=True)
heatmap.set_title('Kendall Correlation Heatmap', fontdict={'fontsize':12}, pad=12);
plt.show()